# **Projeto Jacurutu**
##### O Jacurutu (Corujão-orelhudo) é a maior ave de rapina noturna presente no Brasil, conhecida por sua visão e audição aguçada ela monitora seus alvos antes da caça. 
##### A ideia do projeto é a mesma, monitorar os dados em nossa floresta (Isolation Forest) de forma aguçada para encontrar outliers 🦉

### **A base de dados**
##### A base usada no projeto será a de extrato detalhado dos cartões de crédito corporativos do Governo Federal. Retirada diretamente do Portal da Transparencia, iremos analisar os gastos realziados desde 2023.
- URL de download: https://portaldatransparencia.gov.br/download-de-dados/cpgf
- Dicionário dos dados: https://portaldatransparencia.gov.br/dicionario-de-dados/cpgf

### Nesse notebook iremos fazer a etapa 4.1. Ingestão e Limpeza

In [1]:
#Importações
import pandas as pd
import os
import numpy as np

In [5]:
#Criando o dataset adicionando uma coluna indicando o arquivo original
csv_path = '../raw_data'
csv_files = [f for f in os.listdir(csv_path) if f.lower().endswith('.csv')]
dfs = []

## Fazer uma função
for file in csv_files:
    file_path = os.path.join(csv_path, file)
    df_temp = pd.read_csv(
        file_path,
        sep=';',
        encoding='latin-1',
        low_memory=False
    )
    df_temp['ARQUIVO ORIGEM'] = file
    dfs.append(df_temp)

CPGF_df = pd.concat(dfs, ignore_index=True)

In [ ]:
# Formato função
def injetar_csvs_no_dataframe(csv_path):
    """
    Carrega todos os arquivos CSV do diretório Database CPGF no DataFrame CPGF_df,
    adicionando a coluna 'ARQUIVO ORIGEM' com o nome do arquivo csv original.
    """
    dfs = []
    csv_files = [f for f in os.listdir(csv_path) if f.lower().endswith('.csv')]
    for file in csv_files:
        file_path = os.path.join(csv_path, file)
        df_temp = pd.read_csv(file_path,sep=';',encoding='latin-1',low_memory=False)
        df_temp['ARQUIVO ORIGEM'] = file
        dfs.append(df_temp)
    CPGF_df = pd.concat(dfs, ignore_index=True)
    return CPGF_df

In [ ]:
#Usando a função
CPGF_df = injetar_csvs_no_dataframe(csv_path)

In [ ]:
CPGF_df_limpo = CPGF_df.copy()

In [ ]:
# Analisando a base
CPGF_df_limpo.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392089 entries, 0 to 392088
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   CÓDIGO ÓRGÃO SUPERIOR   392089 non-null  int64 
 1   NOME ÓRGÃO SUPERIOR     392089 non-null  object
 2   CÓDIGO ÓRGÃO            392089 non-null  int64 
 3   NOME ÓRGÃO              392089 non-null  object
 4   CÓDIGO UNIDADE GESTORA  392089 non-null  int64 
 5   NOME UNIDADE GESTORA    392089 non-null  object
 6   ANO EXTRATO             392089 non-null  int64 
 7   MÊS EXTRATO             392089 non-null  int64 
 8   CPF PORTADOR            299427 non-null  object
 9   NOME PORTADOR           392089 non-null  object
 10  CNPJ OU CPF FAVORECIDO  392089 non-null  int64 
 11  NOME FAVORECIDO         392089 non-null  object
 12  TRANSAÇÃO               392089 non-null  object
 13  DATA TRANSAÇÃO          299427 non-null  object
 14  VALOR TRANSAÇÃO         392089 non-n

In [ ]:
CPGF_df_limpo.head()

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,ANO EXTRATO,MÊS EXTRATO,CPF PORTADOR,NOME PORTADOR,CNPJ OU CPF FAVORECIDO,NOME FAVORECIDO,TRANSAÇÃO,DATA TRANSAÇÃO,VALOR TRANSAÇÃO,ARQUIVO ORIGEM
0,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.725.752-**,VIVIANE CORREA LIMA,2179328000142,FREITAS & CIA LTDA,COMPRA A/V - R$ - APRES,21/02/2024,"8,42",202403_CPGF.csv
1,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.866.951-**,JONAS SCHOTTZ DA SILVA,37828985000158,CASA DO SINDICO LTDA,COMPRA A/V - R$ - APRES,20/02/2024,"72,00",202403_CPGF.csv
2,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.866.951-**,JONAS SCHOTTZ DA SILVA,33216995000181,RADEL ELETRONICA LTDA,COMPRA A/V - R$ - APRES,21/02/2024,"26,00",202403_CPGF.csv
3,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.866.951-**,JONAS SCHOTTZ DA SILVA,10421458000178,T-9 ELETRONICA E INFORMATICA LTDA,COMPRA A/V - R$ - APRES,07/02/2024,"1352,00",202403_CPGF.csv
4,63000,Advocacia-Geral da União,63000,Advocacia-Geral da União - Unidades com víncul...,110161,SUPERINTENDENCIA REG. DE ADMIN. DA 1ª REGIAO,2024,3,***.725.752-**,VIVIANE CORREA LIMA,16577772000120,SO FILTROS RONDONIA LTDA,COMPRA A/V - R$ - APRES,21/02/2024,"570,00",202403_CPGF.csv


#### Pelas informações vamos separar a limpeza da base dos dados em 4 pontos:
1. Tratando colunas com dados faltantes
2. Corrigindo os tipos das colunas
3. Analisando e tratando a CNPJ OU CPF FAVORECIDO
4. Analisando e tratando a CPF PORTADOR

#### 1. Tratando dados faltantes

In [ ]:
# Confirmando dados vazios/faltantes
colunas_vazias = (CPGF_df_limpo.isnull().sum() / len(CPGF_df_limpo)) * 100
print(colunas_vazias[colunas_vazias > 0])

CPF PORTADOR      23.6329
DATA TRANSAÇÃO    23.6329
dtype: float64


##### Como a porcentagem de dados faltantes em ambas as colunas são **identicas**, podemos assumir que se tratam das mesmas linhas. Embora possamos imputar o 'CPF do portador' usando a coluna 'nome do portador', infelizmente não conseguimos imputar a data da transação, portanto optei por 'dropar' essas linhas.

In [ ]:
# Excluindo dados faltantes
colunas_drop = ['DATA TRANSAÇÃO', 'CPF PORTADOR']

linhas_antes = len(CPGF_df_limpo)
CPGF_df_limpo = CPGF_df_limpo.dropna(subset=colunas_drop)
linhas_removidas = linhas_antes - len(CPGF_df_limpo)

print(f"{linhas_removidas} linhas foram removidas.")
print(f"Total de linhas final no df_limpo: {len(CPGF_df_limpo)}")

92662 linhas foram removidas.
Total de linhas final no df_limpo: 299427


In [ ]:
# Confirmando a exclusão
colunas_vazias = (CPGF_df_limpo.isnull().sum() / len(CPGF_df_limpo)) * 100
print(colunas_vazias[colunas_vazias > 0])

Series([], dtype: float64)


#### 2. Corrigindo os tipos das colunas

In [ ]:
# Ajustando coluna de Data
CPGF_df_limpo['DATA TRANSAÇÃO'] = pd.to_datetime(CPGF_df_limpo['DATA TRANSAÇÃO'], format='%d/%m/%Y', errors='coerce')

# Ajustando coluna de Valores
CPGF_df_limpo['VALOR TRANSAÇÃO'] = CPGF_df_limpo['VALOR TRANSAÇÃO'].astype(str).str.replace('.', '').str.replace(',', '.')
CPGF_df_limpo['VALOR TRANSAÇÃO'] = pd.to_numeric(CPGF_df_limpo['VALOR TRANSAÇÃO'], errors='coerce').fillna(0)

#### 3. Analisando e tratando a CNPJ OU CPF FAVORECIDO
##### Para tratar a coluna 'CNPJ ou CPF favorecido' de forma consistente e garantir que ela contenha documentos válidos (11 ou 14 dígitos), vamos separar os registros por tamanho e depois aplicar validações específicas para cada tipo. 

In [ ]:
# Verificando todos os tamanhos possíveis
CPGF_df_limpo['CNPJ OU CPF FAVORECIDO'] = CPGF_df_limpo['CNPJ OU CPF FAVORECIDO'].astype(str)
tamanhos = CPGF_df_limpo['CNPJ OU CPF FAVORECIDO'].str.len().value_counts().sort_index()
print(tamanhos)

CNPJ OU CPF FAVORECIDO
2      54207
4          1
6          3
7          9
8         57
9        611
10      4050
11      7254
12      6248
13     73209
14    153778
Name: count, dtype: int64


In [ ]:
tamanhos_problematicos = [2, 4, 6, 7, 8, 9, 10, 12, 13]
serie_tamanhos = CPGF_df_limpo['CNPJ OU CPF FAVORECIDO'].str.len()

for tamanho in tamanhos_problematicos:
    print(f"Valores Mais Comuns para Tamanho = {tamanho}")
    valores_deste_tamanho = CPGF_df_limpo['CNPJ OU CPF FAVORECIDO'][serie_tamanhos == tamanho]
    print(valores_deste_tamanho.value_counts().head(5))

Valores Mais Comuns para Tamanho = 2
CNPJ OU CPF FAVORECIDO
-2    35651
-1    16416
Name: count, dtype: int64
Valores Mais Comuns para Tamanho = 4
CNPJ OU CPF FAVORECIDO
5150    1
Name: count, dtype: int64
Valores Mais Comuns para Tamanho = 6
CNPJ OU CPF FAVORECIDO
320005    1
652504    1
170003    1
Name: count, dtype: int64
Valores Mais Comuns para Tamanho = 7
CNPJ OU CPF FAVORECIDO
8096244    2
2149060    1
1252615    1
9648674    1
2890003    1
Name: count, dtype: int64
Valores Mais Comuns para Tamanho = 8
CNPJ OU CPF FAVORECIDO
58419250    14
93448538     5
61647063     3
12450316     3
25616110     3
Name: count, dtype: int64
Valores Mais Comuns para Tamanho = 9
CNPJ OU CPF FAVORECIDO
997286210    21
793434068    12
419210970    11
646004247    10
658680803    10
Name: count, dtype: int64
Valores Mais Comuns para Tamanho = 10
CNPJ OU CPF FAVORECIDO
4899465181    40
1614356297    35
3668384908    29
3156667226    19
4582352103    18
Name: count, dtype: int64
Valores Mais Comuns pa

##### Pelos dados mais comuns é possível entender que muitos dos dados divergentes estão dessa forma por falta do 0 à esquerda. Exemplo '1060735000173' tem como final um claro comportamento de documento CNPJ /0001-73 faltando somente um zero à esquera para completar o documento.
##### Outro ponto é a coluna **Nome Favorecido** que é a contraparte ao documento onde foi gasto o valor.

##### Com isso vamos usar 2 estratégias para o imputar esses valores:
1. Zeros à esquera: Usaremos nos dados com tamanhos próximos ao correto
2. Investigação a partir dos Nomes: Usaremos nos dados restantes

##### 3.1 Zeros à esquerda

In [ ]:
CPGF_df_limpo['CNPJ OU CPF FAVORECIDO CORRIGIDO'] = CPGF_df_limpo['CNPJ OU CPF FAVORECIDO']

mascara_cpf_fix = (serie_tamanhos >= 8) & (serie_tamanhos <= 10)
mascara_cnpj_fix = (serie_tamanhos == 12) | (serie_tamanhos == 13)

# Padding CPF
CPGF_df_limpo.loc[mascara_cpf_fix, 'CNPJ OU CPF FAVORECIDO CORRIGIDO'] = CPGF_df_limpo['CNPJ OU CPF FAVORECIDO'][mascara_cpf_fix].str.pad(width=11, side='left', fillchar='0')

# Padding CNPJ
CPGF_df_limpo.loc[mascara_cnpj_fix, 'CNPJ OU CPF FAVORECIDO CORRIGIDO'] = CPGF_df_limpo['CNPJ OU CPF FAVORECIDO'][mascara_cnpj_fix].str.pad(width=14, side='left', fillchar='0')

In [ ]:
tamanhos_antigos_dist = serie_tamanhos.value_counts().sort_index()

tamanhos_novos_series = CPGF_df_limpo['CNPJ OU CPF FAVORECIDO CORRIGIDO'].str.len()
tamanhos_novos_dist = tamanhos_novos_series.value_counts().sort_index()

total_problematico_antes = serie_tamanhos[~serie_tamanhos.isin([11, 14])].count()
total_problematico_depois = tamanhos_novos_series[~tamanhos_novos_series.isin([11, 14])].count()
recuperados = total_problematico_antes - total_problematico_depois

print(tamanhos_novos_dist)

print(f"Linhas problemáticas originais (tamanho != 11 ou 14): {total_problematico_antes}")
print(f"Linhas problemáticas restantes (tamanho != 11 ou 14): {total_problematico_depois}")
print(f"LINHAS RECUPERADAS: {recuperados}")

CNPJ OU CPF FAVORECIDO CORRIGIDO
2      52067
4          1
6          3
7          6
11     11323
14    222908
Name: count, dtype: int64
Linhas problemáticas originais (tamanho != 11 ou 14): 132576
Linhas problemáticas restantes (tamanho != 11 ou 14): 52077
LINHAS RECUPERADAS: 80499


##### 3.2 Investigando os nomes dos favorecidos

In [ ]:
coluna_corrigida = 'CNPJ OU CPF FAVORECIDO CORRIGIDO'
coluna_nome = 'NOME FAVORECIDO'

tamanhos_problematicos_restantes = [2, 4, 6, 7]
serie_tamanhos_novos = CPGF_df_limpo[coluna_corrigida].str.len()

dados_incompletos = serie_tamanhos_novos.isin(tamanhos_problematicos_restantes)
df_dados_incompletos = CPGF_df_limpo[dados_incompletos]

print(f"{len(df_dados_incompletos)} linhas sem informação de CNPJ favorecido.")
print(df_dados_incompletos[coluna_nome].value_counts())

52077 linhas sem informação de CNPJ favorecido.
NOME FAVORECIDO
NAO SE APLICA                     35651
SEM INFORMACAO                    16416
BANCO DO BRASIL SA                    4
DILCILEY MATOS DA SILVA               2
CLAUDIO JEFERSON RUBIM MARTINS        1
SUDARIO SEBASTIAO LEAL                1
MARCIA BARROS ALVES ROSA              1
JULIANA OLIVEIRA GHELLERE             1
Name: count, dtype: int64


Manualmente conseguimos encontrar o CNPJ do Banco do Brasil SA que é 00000000000191.

In [ ]:
nome_alvo = 'BANCO DO BRASIL SA'
cnpj_correto = '00000000000191'
coluna_nome = 'NOME FAVORECIDO'
coluna_corrigida = 'CNPJ OU CPF FAVORECIDO CORRIGIDO'

mascara_alvo = (CPGF_df_limpo[coluna_nome] == nome_alvo)
CPGF_df_limpo.loc[mascara_alvo, coluna_corrigida] = cnpj_correto

##### Dos nomes dos dados de favorecidos 2 se destacam pela volumetria e mensagem, NAO SE APLICA e SEM INFORMACAO. Por mais que eles não tenha um CNPJ/CPF direto associado, essas informações são **muito importantes**, pois eles significam que estas transações que não possuem dados diretos de favorecido declarados.
##### Como iremos usar o Isolation Forest para nossa análise futura, já pensando em como vamos classificar cada tipo de feature, iremos criar uma categoria para o favorecido informado em cada uma dessas transações.

##### 3.3 Criando a classificação de favorecido

In [ ]:
condicoes = [(serie_tamanhos == 11), (serie_tamanhos == 14), (serie_tamanhos == 2)]
categorias = ['CPF','CNPJ','NAO_INFORMADO']

CPGF_df_limpo['TIPO FAVORECIDO'] = np.select(condicoes,categorias,default='INVALIDO')

In [ ]:
# Drop nos favorecidos não classificados
linhas_para_dropar = CPGF_df_limpo[CPGF_df_limpo['TIPO FAVORECIDO'] == 'INVALIDO'].index
CPGF_df_limpo = CPGF_df_limpo.drop(linhas_para_dropar)

In [ ]:
# Analisando a nova feature
print("Contagem de linhas por categoria:")
print(CPGF_df_limpo['TIPO FAVORECIDO'] .value_counts())

Contagem de linhas por categoria:
TIPO FAVORECIDO
CNPJ             146881
NAO_INFORMADO     52067
CPF                6851
Name: count, dtype: int64


In [ ]:
# Substituindo a CNPJ OU CPF FAVORECIDO pela CNPJ OU CPF FAVORECIDO CORRIGIDO
CPGF_df_limpo = CPGF_df_limpo.drop(columns=['CNPJ OU CPF FAVORECIDO'])
CPGF_df_limpo = CPGF_df_limpo.rename(columns={'CNPJ OU CPF FAVORECIDO CORRIGIDO':'CNPJ OU CPF FAVORECIDO'})

#### 4. Analisando e tratando a CPF PORTADOR

In [ ]:
tamanhos_cpf_portador = CPGF_df_limpo['CPF PORTADOR'].str.len().value_counts().sort_index()
print(tamanhos_cpf_portador)

CPF PORTADOR
14    205799
Name: count, dtype: int64


##### Como o formato dos CPFs está de acordo com o esperado, não será necessário fazer alterações nesta coluna.

In [ ]:
CPGF_df_limpo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205799 entries, 0 to 375692
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   CÓDIGO ÓRGÃO SUPERIOR   205799 non-null  int64         
 1   NOME ÓRGÃO SUPERIOR     205799 non-null  object        
 2   CÓDIGO ÓRGÃO            205799 non-null  int64         
 3   NOME ÓRGÃO              205799 non-null  object        
 4   CÓDIGO UNIDADE GESTORA  205799 non-null  int64         
 5   NOME UNIDADE GESTORA    205799 non-null  object        
 6   ANO EXTRATO             205799 non-null  int64         
 7   MÊS EXTRATO             205799 non-null  int64         
 8   CPF PORTADOR            205799 non-null  object        
 9   NOME PORTADOR           205799 non-null  object        
 10  NOME FAVORECIDO         205799 non-null  object        
 11  TRANSAÇÃO               205799 non-null  object        
 12  DATA TRANSAÇÃO          205799 non-